In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.feature_selection import VarianceThreshold

%matplotlib inline

## Get the data


```
    Total Points – Bonus Points (tp-bp), Minutes, Yellow Cards, Red Cards, Expected Goals (xG), Expected Assists (xA), Non-penalty Expected Goals (npxG),
    Shots, Expected Goals Against, _Expected_goal_involvements_,  clean_sheets, ict_index, opponent_team, Expected Goals Buildup (xG Buildup), threat, value,
    Key Passes,


    _Games_,  Expected Goals Chain (xG Chain),  _Non-penalty Expected Goal Difference (npxGD)_, _Non-penalty Expected Goals Against (npxGA)_, Expected Points (xPts)
```


In [32]:
columns = [
    'total_points', 'bonus', 'minutes', 'yellow_cards', 'red_cards', 'expected_goals', 'expected_assists', 'npxG', 'shots', 'expected_goal_involvements', 'expected_goals_conceded', 'clean_sheets', 'ict_index',
    'xGBuildup', 'threat', 'value', 'key_passes', 'xGChain', 'xP', 'team_h_difficulty', 'team_a_difficulty', 'was_home'
]

In [36]:
# player_21_22 = pd.read_csv('./data/joint/21-22/merged_player_data.csv')[columns]
player_22_23 = pd.read_csv('./data/joint/22-23/merged_player_data.csv')[columns]
player_23_24 = pd.read_csv('./data/joint/23-24/merged_player_data.csv')[columns]


In [88]:
## Filter out players with zero points
player_22_23_no = player_22_23[player_22_23['total_points'] !=0]
player_23_24_no = player_23_24[player_23_24['total_points'] !=0]

player_data = pd.concat([player_22_23_no, player_23_24_no])

def points_(row):
    return row['total_points'] - row['bonus']
player_data['pts_bps'] = player_data.apply(points_, axis=1)
player_data = player_data.drop(['total_points', 'bonus'], axis=1)

In [89]:
player_data

,minutes,yellow_cards,red_cards,expected_goals,expected_assists,npxG,shots,expected_goal_involvements,expected_goals_conceded,clean_sheets,...,xGBuildup,threat,value,key_passes,xGChain,xP,team_h_difficulty,team_a_difficulty,was_home,pts_bps
0,90,0,0,0.05,0.21,0.054937,1,0.26,1.36,0,...,0.285090,1.0,50,5,0.331344,2.0,2,2,False,2
2,90,0,0,0.05,0.01,0.062027,1,0.06,0.49,1,...,0.067597,8.0,50,0,0.067597,2.2,2,2,False,6
3,71,0,0,0.00,0.01,0.000000,0,0.01,0.44,0,...,0.254193,0.0,49,0,0.254193,1.7,3,3,True,2
4,90,0,0,0.04,0.09,0.035111,1,0.13,0.36,0,...,0.236931,10.0,48,2,0.236931,2.5,2,2,False,2
5,90,1,0,0.00,0.01,0.000000,0,0.01,0.74,1,...,0.204231,0.0,48,1,0.244931,3.5,2,3,True,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7772,45,0,0,0.14,0.03,0.121570,4,0.17,0.95,0,...,0.051355,10.0,50,0,0.114907,2.3,2,4,True,1
7774,26,0,0,0.00,0.02,0.000000,0,0.02,0.11,0,...,0.082490,0.0,50,0,0.082490,2.0,5,4,True,1
7775,62,0,0,0.00,0.04,0.000000,0,0.04,1.66,0,...,0.104135,2.0,50,1,0.126389,2.0,3,3,False,2
7776,90,1,0,0.04,0.04,0.021189,1,0.08,3.22,0,...,0.345725,1.0,50,0,0.345725,1.0,3,5,False,1


In [91]:
player_target = player_data['pts_bps']
player_feat = player_data.drop('pts_bps', axis=1)


## Modeling
